In [3]:
import pandas as pd
import os

export_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\10.25.25\Export"
import_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\10.25.25\COB\Data"
os.makedirs(export_path, exist_ok=True)

category_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\CATEGORY.xlsx")

field_supervisors_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\Field Supervisors.xlsx")

week_df = pd.read_excel(
    r"C:\Users\User\OneDrive\Kimberlin Enterprises\REPORTS\References\WEEK.xlsx")


In [4]:
#INVOICE ON Arrangement

im_filename_inv = "DMS-Invoice-on.xlsx"
full_im_path_inv = os.path.join(import_path, im_filename_inv)

df1 = pd.read_excel(full_im_path_inv)

dfc = df1[['Invoice Date', 'Sold To Customer Number',
'Sold To Customer Name', 'Product Code', 'Product/Item Description',
'Total Item amount with Tax and Discount', 'Invoice Item Type']]
inv = dfc[dfc['Invoice Item Type'] != 'ITM_SALES_TAX']

inv_f = inv
inv_f.insert(6,'BO',0)
inv_f.insert(7,'FG',0)
inv_f['Total Item amount with Tax and Discount'] = inv_f['Total Item amount with Tax and Discount'].fillna(0)
inv_f = inv_f.drop('Invoice Item Type', axis=1)

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_13520\1227273928.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inv_f['Total Item amount with Tax and Discount'] = inv_f['Total Item amount with Tax and Discount'].fillna(0)


In [5]:
#CUSTOMER RETURNS Arrangement

im_filename_ret = "DMS-Customer Returns-on.xlsx"
full_im_path_ret = os.path.join(import_path, im_filename_ret)

df2 = pd.read_excel(full_im_path_ret)

dfc2 = df2[['Customer Return Date', 'Sold To Customer Number', 'Sold To Customer Name', 
'Product Code', 'Product Description', 'Facility Name', 'Estimated Product Return Amount']]
dfc2['Estimated Product Return Amount'].fillna(0, inplace=True)
dfc2.insert(5,'Total Item amount with Tax and Discount', 0)
dfc2.insert(8,'with vat', 0)
dfc2['with vat'] = dfc2['Estimated Product Return Amount'] * 1.12

cust_ret = dfc2.pivot_table(index=['Customer Return Date', 'Sold To Customer Number', 'Sold To Customer Name', 
'Product Code', 'Product Description', 'Total Item amount with Tax and Discount'], columns='Facility Name', values='with vat', aggfunc=sum)

ret_f = cust_ret.reset_index()
ret_f['BO'] = ret_f['BO'].fillna(0)
ret_f['FG'] = ret_f['FG'].fillna(0)

C:\Users\User\AppData\Local\Temp\ipykernel_13520\3676257053.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfc2['Estimated Product Return Amount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13520\3676257053.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc2['Estimated Product Return Amount'].fillna(0, inpla

In [6]:
#Customer Master List

im_filename_cust = "DMS-Customer-on.xlsx"
full_im_path_cust = os.path.join(import_path, im_filename_cust)

cust_df = pd.read_excel(full_im_path_cust)

cust_df = cust_df[[
    'NEXT_UP_NUMBER',
    'PARTY_CLASSIFICATION_DESCRIPTION',
    'KEY_ACCOUNT',
    'SALES_REP_ID',
    'SALES_REP_NAME',
    'BRANCH_NAME',
    'GEO_LOCATION_HIERARCHYDESCRIPTION',
    'CITY',
    'STATE_PROVINCE',
    'CHANNEL'
]]

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
#COMBINE INVOICED AND RETURNS to NET INVOICED

inv_f.rename(columns={
'Invoice Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE',
'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE',
'Product/Item Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

ret_f.rename(columns={
'Customer Return Date': 'DATE',
'Sold To Customer Number': 'ACCOUNT CODE', 
'Sold To Customer Name': 'ACCOUNT NAME',
'Product Code': 'SKU CODE', 
'Product Description': 'SKU NAME',
'Total Item amount with Tax and Discount': 'SERVED INVOICE',
'BO': 'BAD RETURNS',
'FG': 'GOOD RETURNS'
}, inplace=True)

net_inv = pd.concat([inv_f, ret_f], axis=0)

ex_filename_net_inv = "Net_Invoiced.xlsx"
full_path_net_inv = os.path.join(export_path, ex_filename_net_inv)

cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'ACCOUNT CODE'
})

net_inv_f = net_inv.merge(cml_df, on='ACCOUNT CODE', how='left')

net_inv_f.head()
# export_net_inv = net_inv_f
# export_net_inv = export_net_inv.sort_values(by=['DATE', 'ACCOUNT CODE'])
# export_net_inv.to_excel(full_path_net_inv, index=False)

,DATE,ACCOUNT CODE,ACCOUNT NAME,SKU CODE,SKU NAME,SERVED INVOICE,BAD RETURNS,GOOD RETURNS,PARTY_CLASSIFICATION_DESCRIPTION,KEY_ACCOUNT,SALES_REP_ID,SALES_REP_NAME,BRANCH_NAME,GEO_LOCATION_HIERARCHYDESCRIPTION,CITY,STATE_PROVINCE,CHANNEL
0,10/18/2025,3115000486,ROSE AFRICA STORE,2759,C2 Green Tea Apple 335mlx24,1153.00,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,SAN RAFAEL,GUIMBA,NUEVA ECIJA,VAN(EXTRUCK)
1,10/18/2025,3115000486,ROSE AFRICA STORE,6801,C2 SOLO APPLE SW 230ML x 24,599.76,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,SAN RAFAEL,GUIMBA,NUEVA ECIJA,VAN(EXTRUCK)
2,10/14/2025,3118000313,LAILAS STORE TALUGTUG,55211,NISSIN CUP MINI SEAFOOD 40g x 48,41.93,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,QUEZON (POB.),TALUGTUG,NUEVA ECIJA,VAN(EXTRUCK)
3,10/14/2025,3118000313,LAILAS STORE TALUGTUG,6801,C2 SOLO APPLE SW 230ML x 24,299.88,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,QUEZON (POB.),TALUGTUG,NUEVA ECIJA,VAN(EXTRUCK)
4,10/14/2025,3118000313,LAILAS STORE TALUGTUG,10574,CHIZ CURLS 18g x 100,34.27,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,QUEZON (POB.),TALUGTUG,NUEVA ECIJA,VAN(EXTRUCK)


In [18]:
category_df = category_df.drop(['SKU NAME'], axis=1)
net_inv_f_l1 = net_inv_f.merge(category_df,  on='SKU CODE', how='left')

net_inv_f_l2 = net_inv_f_l1.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

net_inv_f_l3 = net_inv_f_l2.merge(week_df, on='DATE', how='inner')

net_inv_f_l3.head()

,DATE,ACCOUNT CODE,ACCOUNT NAME,SKU CODE,SKU NAME,SERVED INVOICE,BAD RETURNS,GOOD RETURNS,PARTY_CLASSIFICATION_DESCRIPTION,KEY_ACCOUNT,SALES_REP_ID,SALES_REP_NAME,BRANCH_NAME,GEO_LOCATION_HIERARCHYDESCRIPTION,CITY,STATE_PROVINCE,CHANNEL,CATEGORY,FS,WEEK
0,10/18/2025,3115000486,ROSE AFRICA STORE,2759,C2 Green Tea Apple 335mlx24,1153.00,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,SAN RAFAEL,GUIMBA,NUEVA ECIJA,VAN(EXTRUCK),Pet Tea,JAYCO JUSI,Week 3
1,10/18/2025,3115000486,ROSE AFRICA STORE,6801,C2 SOLO APPLE SW 230ML x 24,599.76,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,SAN RAFAEL,GUIMBA,NUEVA ECIJA,VAN(EXTRUCK),Pet Tea,JAYCO JUSI,Week 3
2,10/14/2025,3118000313,LAILAS STORE TALUGTUG,55211,NISSIN CUP MINI SEAFOOD 40g x 48,41.93,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,QUEZON (POB.),TALUGTUG,NUEVA ECIJA,VAN(EXTRUCK),NUR,JAYCO JUSI,Week 3
3,10/14/2025,3118000313,LAILAS STORE TALUGTUG,6801,C2 SOLO APPLE SW 230ML x 24,299.88,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,QUEZON (POB.),TALUGTUG,NUEVA ECIJA,VAN(EXTRUCK),Pet Tea,JAYCO JUSI,Week 3
4,10/14/2025,3118000313,LAILAS STORE TALUGTUG,10574,CHIZ CURLS 18g x 100,34.27,0.0,0.0,Sari-sari Stores - X,Sari-Sari Stores,KNE0015,GERENCE MARTIN .,KNE,QUEZON (POB.),TALUGTUG,NUEVA ECIJA,VAN(EXTRUCK),Snacks,JAYCO JUSI,Week 3


In [23]:
#Price List M0

im_filename_pl = "DMS-Price-on.xlsx"
full_im_path_pl = os.path.join(import_path, im_filename_pl)

pl_df = pd.read_excel(full_im_path_pl)

pl_df2 = pl_df

pl_df2 = pl_df2[['product_code', 'product_description', 'uom_description', 'selling_price', 'cust_class', 'cust_channel']]
pl_df2.insert(4,'with vat', 0)
pl_df2['with vat'] = pl_df2['selling_price'] * 1.12

pl_df_m0 = pl_df2

pl_df_m0 = pl_df_m0[pl_df_m0['cust_class'] != 'BEV Dealer']
pl_df_m0 = pl_df_m0[pl_df_m0['cust_channel'] != 'VAN(EXTRUCK)']

pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description',], columns='uom_description', values='with vat', aggfunc=sum)

pl_df_m0_f_1 = pl_df_m0_f.reset_index()

pl_df_m0_f_1['Case'] = pl_df_m0_f_1['Case'].fillna(0)
pl_df_m0_f_1['Piece'] = pl_df_m0_f_1['Piece'].fillna(0)
pl_df_m0_f_1['Subcase'] = pl_df_m0_f_1['Subcase'].fillna(0)

pl_m0_final = pl_df_m0_f_1[['product_code', 'product_description', 'Case', 'Subcase', 'Piece']]

##################################

pl_m0_final_reference = pl_m0_final[['product_code', 'Case']]

pl_m0_final_reference = pl_m0_final_reference.rename(columns={
    'product_code': 'SKU CODE',
    'Case': 'SKU PRICE REFERENCE'
})

net_inv_f_l4 = net_inv_f_l3.merge(pl_m0_final_reference, on='SKU CODE', how='left')

net_inv_f_l4.insert(5,'VALUE', 0)
net_inv_f_l4['VALUE'] = net_inv_f_l4['SERVED INVOICE'] - net_inv_f_l4['BAD RETURNS'] - net_inv_f_l4['GOOD RETURNS']

# ensure numeric and avoid divide-by-zero when computing VOLUME
net_inv_f_l4['SKU PRICE REFERENCE'] = pd.to_numeric(net_inv_f_l4['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
net_inv_f_l4['VOLUME'] = 0
mask = net_inv_f_l4['SKU PRICE REFERENCE'] != 0
net_inv_f_l4.loc[mask, 'VOLUME'] = net_inv_f_l4.loc[mask, 'VALUE'] / net_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']



c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_13520\3454206891.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_df2['with vat'] = pl_df2['selling_price'] * 1.12
C:\Users\User\AppData\Local\Temp\ipykernel_13520\3454206891.py:19: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description

In [ ]:
# Add before exporting to Excel
ex_filename_net_inv = "Net_Invoiced_samlpe_10.28.25_4.xlsx"
full_path_net_inv = os.path.join(export_path, ex_filename_net_inv)

export_net_inv = net_inv_f_l4
export_net_inv['RD NAME'] = 'Kimberlin'

export_net_inv = export_net_inv.sort_values(by=['DATE', 'ACCOUNT CODE'])

export_net_inv.rename(columns={
    'RD NAME': 'RD Name',
    'DATE': 'Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'ACCOUNT CODE': 'Sold To Customer number',
    'ACCOUNT NAME': 'Sold To Customer Name',
    'CATEGORY': 'Category',
    'SKU CODE': 'Product Code',
    'SKU NAME': 'Product Description',
    'VOLUME': 'Volume',
    'VALUE': 'Net Value',
    'GOOD RETURNS': 'Good Stock Returns',
    'BAD RETURNS': 'Bad Stock Returns',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel_Classification',
    'GEO_LOCATION_HIERARCHYDESCRIPTION': 'Brgy',
    'CITY': 'Town',
    'STATE_PROVINCE': 'Province',
    'FS': 'FS',
    'CHANNEL': 'RTM Model'
}, inplace=True)

export_net_inv_final = export_net_inv[[
    'RD Name',
    'Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Net Value',
    'Good Stock Returns',
    'Bad Stock Returns',
    'Channel_Classification',
    'Brgy',
    'Town',
    'Province',
    'FS',
    'RTM Model',
]]

export_net_inv_final.to_excel(full_path_net_inv, index=False)

In [39]:
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, NamedStyle
from openpyxl.utils import get_column_letter

# Step 1: Export to Excel with a blank row at the top
temp_output = 'styled_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    export_net_inv_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 17):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 17):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N', 'O', 'P']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding


# Save changes
Net_Invoiced = "Net_Invoiced_samlpe_10.25.25_2.xlsx"
Net_Invoiced_path = os.path.join(export_path, Net_Invoiced)

ws.sheet_view.showGridLines = False
wb.save(Net_Invoiced_path)


In [41]:
inv_f.head()

,DATE,ACCOUNT CODE,ACCOUNT NAME,SKU CODE,SKU NAME,SERVED INVOICE,BAD RETURNS,GOOD RETURNS
0,10/18/2025,3115000486,ROSE AFRICA STORE,2759,C2 Green Tea Apple 335mlx24,1153.00,0,0
2,10/18/2025,3115000486,ROSE AFRICA STORE,6801,C2 SOLO APPLE SW 230ML x 24,599.76,0,0
4,10/14/2025,3118000313,LAILAS STORE TALUGTUG,55211,NISSIN CUP MINI SEAFOOD 40g x 48,41.93,0,0
6,10/14/2025,3118000313,LAILAS STORE TALUGTUG,6801,C2 SOLO APPLE SW 230ML x 24,299.88,0,0
8,10/14/2025,3118000313,LAILAS STORE TALUGTUG,10574,CHIZ CURLS 18g x 100,34.27,0,0


In [73]:
ser_inv = inv_f
ser_inv_df = ser_inv.drop(['BAD RETURNS', 'GOOD RETURNS'], axis=1)

cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'ACCOUNT CODE'
})

ser_inv_df1 = ser_inv_df.merge(cml_df, on='ACCOUNT CODE', how='left')

ser_inv_f_l1 = ser_inv_df1.merge(category_df,  on='SKU CODE', how='left')

ser_inv_f_l2 = ser_inv_f_l1.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

ser_inv_f_l3 = ser_inv_f_l2.merge(week_df, on='DATE', how='inner')

ser_inv_f_l4 = ser_inv_f_l3.merge(pl_m0_final_reference, on='SKU CODE', how='left')

ser_inv_f_l4['SKU PRICE REFERENCE'] = pd.to_numeric(ser_inv_f_l4['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
ser_inv_f_l4['VOLUME'] = 0
mask = ser_inv_f_l4['SKU PRICE REFERENCE'] != 0
ser_inv_f_l4.loc[mask, 'VOLUME'] = ser_inv_f_l4.loc[mask, 'SERVED INVOICE'] / ser_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']

ser_inv_f_l4['RD NAME'] = 'Kimberlin'


C:\Users\User\AppData\Local\Temp\ipykernel_13520\1653891455.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.0399974  2.04       0.04250832 ... 0.49990125 0.99999907 0.9999987 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ser_inv_f_l4.loc[mask, 'VOLUME'] = ser_inv_f_l4.loc[mask, 'SERVED INVOICE'] / ser_inv_f_l4.loc[mask, 'SKU PRICE REFERENCE']


In [80]:

export_ser_inv = ser_inv_f_l4

export_ser_inv.rename(columns={
    'RD NAME': 'RD Name',
    'DATE': 'Invoice Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'ACCOUNT CODE': 'Sold To Customer Number',
    'ACCOUNT NAME': 'Sold To Customer Name',
    'CATEGORY': 'Category',
    'SKU CODE': 'Product Code',
    'SKU NAME': 'Product Description',
    'VOLUME': 'Volume',
    'SERVED INVOICE': 'Value',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel Type'
}, inplace=True)

export_ser_inv_final = export_ser_inv[[
    'RD Name',
    'Invoice Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer Number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Value',
    'FS',
    'Channel Type'
]]

ser_inv_df_export = export_ser_inv_final
ser_inv_df_export_final = ser_inv_df_export.sort_values(by=['Invoice Date', 'Sold To Customer Number'])

# SERVED INVOICE Finalization

# Step 1: Export to Excel with a blank row at the top
temp_output = 'temp_file_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    ser_inv_df_export_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 15):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 15):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save changes
ex_filename_ser_inv = "Served Invoice.xlsx"
full_path_ser_inv = os.path.join(export_path, ex_filename_ser_inv)

ws.sheet_view.showGridLines = False
wb.save(full_path_ser_inv)




In [81]:
#SALES ORDERS

im_filename_so = "DMS-Sales Order-on.xlsx"
full_im_path_so = os.path.join(import_path, im_filename_so)

df3 = pd.read_excel(full_im_path_so)

dfc3 = df3[['Last Modified Date', 'Sold To Customer number', 'Sold To Customer Name',
 'Product Code', 'Product Description', 'Total Product Amount', 'SO status']]
so_par = dfc3[dfc3['SO status'] == 'Invoiced']

so_par['Total Product Amount'].fillna(0, inplace=True)
so_par.insert(6,'with vat', 0)
so_par['with vat'] = so_par['Total Product Amount'] * 1.12

so_f = so_par
so_f = so_f.drop(['Total Product Amount', 'SO status'], axis=1)

cml_df = cust_df.rename(columns={
    'NEXT_UP_NUMBER': 'Sold To Customer number'
})
cat_df = category_df.rename(columns={
    'SKU CODE': 'Product Code'
})
wk_df = week_df.rename(columns={
    'DATE': 'Last Modified Date'
})
pl_df = pl_m0_final_reference.rename(columns={
    'SKU CODE': 'Product Code'
})

sales_orders_df1 = so_f.merge(cml_df, on='Sold To Customer number', how='left')

sales_orders_df2 = sales_orders_df1.merge(cat_df,  on='Product Code', how='left')

sales_orders_df3 = sales_orders_df2.merge(field_supervisors_df, on='SALES_REP_ID', how='left')

sales_orders_df4 = sales_orders_df3.merge(wk_df, on='Last Modified Date', how='inner')

sales_orders_df5 = sales_orders_df4.merge(pl_df, on='Product Code', how='left')

sales_orders_df5['SKU PRICE REFERENCE'] = pd.to_numeric(sales_orders_df5['SKU PRICE REFERENCE'], errors='coerce').fillna(0)
sales_orders_df5['VOLUME'] = 0
mask = sales_orders_df5['SKU PRICE REFERENCE'] != 0
sales_orders_df5.loc[mask, 'VOLUME'] = sales_orders_df5.loc[mask, 'with vat'] / sales_orders_df5.loc[mask, 'SKU PRICE REFERENCE']

sales_orders_df5['RD NAME'] = 'Kimberlin'

sales_orders_df5.head()

c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_13520\571522944.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  so_par['Total Product Amount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13520\571522944.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

,Last Modified Date,Sold To Customer number,Sold To Customer Name,Product Code,Product Description,with vat,PARTY_CLASSIFICATION_DESCRIPTION,KEY_ACCOUNT,SALES_REP_ID,SALES_REP_NAME,...,GEO_LOCATION_HIERARCHYDESCRIPTION,CITY,STATE_PROVINCE,CHANNEL,CATEGORY,FS,WEEK,SKU PRICE REFERENCE,VOLUME,RD NAME
0,10/02/2025,D3114A0018,JOHN CLAIRE,7707,CREAM-O CHOCO CHIP COOKIES 36g x 10 x 20,962.1248,Public Market Reseller,Market Stalls,KNE0006,BERNARDO DELA CRUZ .,...,POBLACION SUR,TALAVERA,NUEVA ECIJA,BOOK(Booking),Biscuits,NILO E. EVANGELISTA,Week 1,1718.0016,0.560026,Kimberlin
1,10/02/2025,D3114A0018,JOHN CLAIRE,2153,Great Taste Granules Strong 40g SUP,2623.2752,Public Market Reseller,Market Stalls,KNE0006,BERNARDO DELA CRUZ .,...,POBLACION SUR,TALAVERA,NUEVA ECIJA,BOOK(Booking),Coffee,NILO E. EVANGELISTA,Week 1,1171.1056,2.239999,Kimberlin
2,10/02/2025,D3114A0018,JOHN CLAIRE,2154,Great Taste Premium Classic 40g SUP,3849.0816,Public Market Reseller,Market Stalls,KNE0006,BERNARDO DELA CRUZ .,...,POBLACION SUR,TALAVERA,NUEVA ECIJA,BOOK(Booking),Coffee,NILO E. EVANGELISTA,Week 1,1145.5584,3.360005,Kimberlin
3,10/02/2025,D3114A0018,JOHN CLAIRE,60325,POT CHIPS V-CUT CHEESE 25g x 80,350.4816,Public Market Reseller,Market Stalls,KNE0006,BERNARDO DELA CRUZ .,...,POBLACION SUR,TALAVERA,NUEVA ECIJA,BOOK(Booking),Snacks,NILO E. EVANGELISTA,Week 1,1251.8800,0.279964,Kimberlin
4,10/02/2025,D3114A0018,JOHN CLAIRE,62553,POT CHIPS V-CUT BBQ 60g x 50,386.1088,Public Market Reseller,Market Stalls,KNE0006,BERNARDO DELA CRUZ .,...,POBLACION SUR,TALAVERA,NUEVA ECIJA,BOOK(Booking),Snacks,NILO E. EVANGELISTA,Week 1,1723.4560,0.224032,Kimberlin


In [83]:
export_sales_order = sales_orders_df5

export_sales_order.rename(columns={
    'RD NAME': 'RD Name',
    'Last Modified Date': 'SO Date',
    'WEEK': 'Week',
    'BRANCH_NAME': 'Branch Name',
    'SALES_REP_ID': 'Employee Code',
    'SALES_REP_NAME': 'Employee Name',
    'KEY_ACCOUNT': 'Channel',
    'Sold To Customer number': 'Sold To Customer Number',
    'CATEGORY': 'Category',
    'VOLUME': 'Volume',
    'with vat': 'Value',
    'PARTY_CLASSIFICATION_DESCRIPTION': 'Channel Type'
}, inplace=True)

export_sales_order_final = export_sales_order[[
    'RD Name',
    'SO Date',
    'Week',
    'Branch Name',
    'Employee Code',
    'Employee Name',
    'Channel',
    'Sold To Customer Number',
    'Sold To Customer Name',
    'Category',
    'Product Code',
    'Product Description',
    'Volume',
    'Value',
    'FS',
    'Channel Type'
]]

so_df_export = export_sales_order_final
so_df_export_final = so_df_export.sort_values(by=['SO Date', 'Sold To Customer Number'])

# SERVED INVOICE Finalization

# Step 1: Export to Excel with a blank row at the top
temp_output = 'temp_file_output.xlsx'
temp_file = os.path.join(import_path, temp_output)

with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
    # Write to row 2 (startrow=1) to leave row 1 blank
    so_df_export_final.to_excel(writer, index=False, startrow=1)

# Step 2: Load workbook and apply formatting
wb = load_workbook(temp_file)
ws = wb.active

# Step 3: Style header (row 2)
header_fill = PatternFill(start_color='000000', end_color='000000', fill_type='solid')
header_font = Font(bold=True, color='FFFFFF')
header_alignment = Alignment(horizontal='center')

for cell in ws[2]:
    cell.fill = header_fill
    cell.font = header_font
    cell.alignment = header_alignment

# Step 4: Format numeric columns with comma style
accounting_style = NamedStyle(name="accounting_style", number_format='_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)')
for col in range(13, 15):  # Assuming numeric columns start from column 2
    for row in range(3, ws.max_row + 1):  # Data starts from row 3
        cell = ws.cell(row=row, column=col)
        if isinstance(cell.value, (int, float)):
            cell.style = accounting_style

sum_row = ws.max_row + 1
start_range = 3
end_range = ws.max_row

for col in range(13, 15):
    col_letter = get_column_letter(col)
    sum_cell = ws.cell(row=sum_row, column=col)
    sum_cell.value = f"=SUM({col_letter}{start_range}:{col_letter}{end_range})"
    sum_cell.font = Font(bold=True, color="FF0000")
    sum_cell.number_format = '_(* #,##0.00_);_(* (#,##0.00);_(* "-"_);_(@_)'

# List the column letters or indexes you want to auto-fit
columns_to_adjust = ['M', 'N']  # or use [2, 4] for indexes

for col_id in columns_to_adjust:
    # Convert index to letter if needed
    col_letter = get_column_letter(col_id) if isinstance(col_id, int) else col_id
    max_length = 0

    for cell in ws[col_letter]:
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save changes
ex_filename_so = "Sales_Orders.xlsx"
full_path_so = os.path.join(export_path, ex_filename_so)

ws.sheet_view.showGridLines = False
wb.save(full_path_so)